# AI Controller: PromptLib

PromptLib is a library for creating AST runner programs for execution by the AICI AST VM. In addition to wrapping AICI AST functionality, PromptLib also adds higher level functionality that it compiles down to AST runner's primitives.

In this tutorial notebook, we'll show how to go over the basic usage of PromptLib, including prompting and generation, constrained generation with regexes and context free grammars,  branches, variables, and control flow.


## Setup

Please see the AICI README to install and run an AICI-enabled LLM service.

In [1]:
import promptlib as pl
import json

AICI_API_BASE=""

# PromptLib currently uses the DeclCtrl, the Declarative Controller
aici = pl.AICI(base_url=AICI_API_BASE, wasm_runner_id="declctrl-latest")

## Basic Usage


We'll begin by demonstrating how to run a simple PromptLib text generation program.

We'll create a PromptProgram object, create a prompt by appending static strings to it, and then we'll append instructions to generate text.

When we run the program, we'll see the log of running the program, followed by '[DONE]'

In [2]:
# Create a new program and set its endpoint to our AICI node
program = pl.PromptProgram(aici)

# A PromptLib program is built up by appending fixed text, generation
# requests, and other commands
p = program + "Please answer the following simple question:\n"
p += "Q: What is the meaning of life?\nA:"
p = p.gen(max_tokens = 20, stop_at="\n")

# Once we've finished building the program, we can run it. 
result = program.run()


Please answer the following simple question:
Q: What is the meaning of life?
A: The meaning of life is to find your purpose.
[DONE]


Results are returned in a structured format, so we can inspect the result variable to see the generated text.

In [3]:
# result['text'] is the full generated text
print(result['text'])

['Please answer the following simple question:\nQ: What is the meaning of life?\nA: The meaning of life is to find your purpose.\n']


In addition to the generated text, the result also includes the original request sent to the server, the detailed response, logs, and other information. 

In [4]:
print("The original request is:\n")
print(json.dumps(result['request'], indent=4))

print("\n\nThe full response is:\n")
print(json.dumps(result['response'], indent=4))

print("\n\nOther sections of information in the response include: " + str(result.keys()))

The original request is:

{
    "model": "",
    "prompt": "",
    "max_tokens": 200,
    "n": 1,
    "temperature": 0,
    "stream": true,
    "aici_module": "declctrl-latest",
    "aici_arg": {
        "steps": [
            {
                "Fixed": {
                    "text": {
                        "String": {
                            "str": "Please answer the following simple question:\n"
                        }
                    },
                    "tag": "_1",
                    "following": null,
                    "label": "_1"
                }
            },
            {
                "Fixed": {
                    "text": {
                        "String": {
                            "str": "Q: What is the meaning of life?\nA:"
                        }
                    },
                    "tag": "_2",
                    "following": null,
                    "label": "_2"
                }
            },
            {
                "Gen": {

Instead of generating arbitrary text, we can also restrict the generation in various ways.  The simplest is to force the language model to choose between two or more options.  The model will then choose the most likely of the options.

In [5]:
program = pl.PromptProgram(aici)

p = program + "Please answer the following simple question:\n"
p += "Q: What is the best television program ever made?\n"

# Force the LLM to choose between the listed options
p = p.choose(["MacGyver", "Star Trek", "She-Ra"])

result = program.run()

Please answer the following simple question:
Q: What is the best television program ever made?
She-Ra[DONE]


We can save the selected choice to a variable.  In fact, we can store the text generated by any gen() command to a variable.

In [6]:
program = pl.PromptProgram(aici)

p = program + "Please answer the following simple question:\n"
p += "Q: What is the best television program ever made?\n"

# We'll add a second argument that will copy the chosen text into a variable for easy access
p = p.choose(["MacGyver", "Star Trek", "She-Ra"], set_var="best_tv_show")

result = program.run()

# Results are returned in a structured representation
print("The answer to the question was: " + result['storage']['best_tv_show'])

Please answer the following simple question:
Q: What is the best television program ever made?
She-Ra[DONE]
The answer to the question was: She-Ra


## Constrained Generation

PromptLib supports several ways to constrain text generation, using regular expressions and context free grammars.

In [7]:
# fixed generation with regex constraints (and see more detailed regex notebook)

program = pl.PromptProgram(aici)

p = program + "Please guess a number: "

# pass in a regular expression (rx) that forces the LLM to generate a number
p = p.gen(rx = r"'\d+'", set_var="number")

result = program.run()

print("The LLM guessed " + result['storage']['number'])


Please guess a number: '10'[DONE]
The LLM guessed '10'


We can combine fixed text with choices and regexes to generate text that matches a specific pattern, like JSON.

In [8]:
program = pl.PromptProgram(aici)

note = """
The 3 astronauts who flew to the International Space Station reached their destination today (May 31). The
crew successfully docked at 10:16 a.m. EDT, about 19 hours after blasting off from
NASA's Kennedy Space Center. The docking went smoothly, with the Crew Dragon's soft capture docking
mechanism attaching automatically at 10:16 a.m. EDT."""

p = program + "Please summarize and extract key factors from the following text:\n"
p += "<text>" + note + "</text>\n"
p += "Return as JSON:"
p += "{shorttitle: '"
p = p.gen(set_var="title") + "', who: '"
p = p.gen(set_var="who") + "', what: '"
p = p.gen(set_var="what") + "', when: '"
p = p.gen(set_var="when") + "', where: '"
p = p.gen(set_var="where") + "', excitement_level: '"
p = p.gen(rx = r"\d+", set_var="excitement_level") + "', quality_level: '"
p = p.choose(["high", "medium", "low"], set_var="quality_level") + "'}\n"

result = program.run()

# print everything in generated text after the 'JSON:'
print(result['text'][0].split('JSON:')[1])


Please summarize and extract key factors from the following text:
<text>
The 3 astronauts who flew to the International Space Station reached their destination today (May 31). The
crew successfully docked at 10:16 a.m. EDT, about 19 hours after blasting off from
NASA's Kennedy Space Center. The docking went smoothly, with the Crew Dragon's soft capture docking
mechanism attaching automatically at 10:16 a.m. EDT.</text>
Return as JSON:{shorttitle: 'Astronauts Reach Space Station', who: '3 astronauts', what: 'reached their destination', when: 'today (May 31)', where: 'International Space Station', excitement_level: '10', quality_level: 'high'}
[DONE]
{shorttitle: 'Astronauts Reach Space Station', who: '3 astronauts', what: 'reached their destination', when: 'today (May 31)', where: 'International Space Station', excitement_level: '10', quality_level: 'high'}



Here's a simple constrained generation example using a context-free grammar.

In [9]:
program = pl.PromptProgram(aici)

# load a grammar file
c_grammar = open("aici_abi/grammars/c.y", "r").read()

p = program + "Please write a C program that recursively calculates the Fibonnaci sequence to N digits.\n"
p += "#include<stdio.h>\n"
p = p.gen(max_tokens = 200, yacc=c_grammar, set_var="code", stop_at='\n\n')

result = program.run()

print(result['storage']['code'])

Please write a C program that recursively calculates the Fibonnaci sequence to N digits.
#include<stdio.h>
int fib(int n)
{
    if(n==0)
        return 0;
    else if(n==1)
        return 1;
    else
        return fib(n-1)+fib(n-2);
}
int main()
{
    int n;
    printf("Enter the number of digits: ");
    scanf("%d",&n);
    printf("The Fibonnaci sequence is: %d",fib(n));
    return 0;
}

[DONE]
int fib(int n)
{
    if(n==0)
        return 0;
    else if(n==1)
        return 1;
    else
        return fib(n-1)+fib(n-2);
}
int main()
{
    int n;
    printf("Enter the number of digits: ");
    scanf("%d",&n);
    printf("The Fibonnaci sequence is: %d",fib(n));
    return 0;
}




For more examples of regular expressions and context free grammars see the detailed constraints tutorials.